# Анализ данных розничных продаж

## 1. Подготовка данных

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from scipy import stats

sns.set(style="whitegrid")

In [ ]:
df_raw = pd.read_csv("retail_sales_dataset.csv")

In [ ]:
df_raw.head()

In [ ]:
df_raw.info()

In [ ]:
df_raw.shape

Датасет содержит информацию о транзакциях, клиентах и покупках в розничном магазине.

In [ ]:
df = df_raw.copy()

In [ ]:
df.columns = (
    df.columns
    .str.strip()          
    .str.lower()          
    .str.replace(" ", "_") 
)

df.columns

In [ ]:
df["date"] = pd.to_datetime(df["date"])
df["total_amount"] = df["total_amount"].astype(float)

In [ ]:
df.dtypes

In [ ]:
df.isnull().sum()

In [ ]:
df.duplicated().sum() ## Проверка дубликатов

Пропущенных значений не обнаружено

In [ ]:
df = df.drop_duplicates()

In [ ]:
df.describe()

In [ ]:
plt.figure(figsize=(5,3))
sns.boxplot(x=df["age"])
plt.title("Boxplot возраста")
plt.show()

In [ ]:
plt.figure(figsize=(5,3))
sns.boxplot(x=df["total_amount"])
plt.title("Boxplot суммы покупки")
plt.show()

Отрицательных значений возраста или цены не обнаружено. Явных аномалий нет.

In [ ]:
plt.figure(figsize=(5,3))
sns.boxplot(x=df["total_amount"])
plt.title("Boxplot суммы покупки")
plt.show()

In [ ]:
df["gender"].unique()

In [ ]:
df["product_category"].unique()

Опечаток и неконсистентных значений не выявлено.

На основе первичного анализа можно сформулировать гипотезы: средний чек отличается между мужчинами и женщинами; категория продукта влияет на общую сумму продаж; пользователи определённых возрастных групп тратят больше.

## 2. Метрики и визуализация

In [ ]:
total_revenue = df["total_amount"].sum()
avg_check = df["total_amount"].mean()
transactions_count = len(df)
unique_customers = df["customer_id"].nunique()

print("Общая выручка:", total_revenue)
print("Средний чек:", avg_check)
print("Количество транзакций:", transactions_count)
print("Уникальные клиенты:", unique_customers)

In [ ]:
sales_by_category = df.groupby("product_category")["total_amount"].sum()
sales_by_category

In [ ]:
sales_by_category.plot(kind="bar", figsize=(6,4))
plt.title("Продажи по категориям товаров")
plt.ylabel("Total Amount")
plt.show()

In [ ]:
df["age_group"] = pd.cut(
    df["age"],
    bins=[0, 25, 35, 50, 100],
    labels=["<25", "25-35", "35-50", ">50"]
)

In [ ]:
df.groupby("age_group", observed=True)["total_amount"].sum()

In [ ]:
plt.figure(figsize=(6,4))
sns.histplot(df["age"], bins=30)
plt.title("Распределение возраста клиентов")
plt.show()

In [ ]:
plt.figure(figsize=(6,4))
sns.boxplot(x="gender", y="total_amount", data=df)
plt.title("Средний чек по полу")
plt.show()

In [ ]:
plt.figure(figsize=(6,4))
sns.scatterplot(x="age", y="total_amount", data=df)
plt.title("Возраст и сумма покупки")
plt.show()

Выводы: наибольшая выручка приходится на категорию Electronics; основной вклад в продажи вносят клиенты в возрасте 25–50 лет; средний чек у женщин немного выше; явной линейной зависимости между возрастом и суммой покупки нет.

## 3. A/B тест

H0: Средний чек у мужчин и женщин одинаков

H1: Средний чек отличается между мужчинами и женщинами

In [ ]:
male = df[df["gender"] == "Male"]["total_amount"]
female = df[df["gender"] == "Female"]["total_amount"]

In [ ]:
print("Средний чек (мужчины):", male.mean())
print("Средний чек (женщины):", female.mean())

In [ ]:
t_stat, p_value = stats.ttest_ind(male, female, equal_var=False)
p_value

In [ ]:
u_stat, p_value_mw = stats.mannwhitneyu(
    male,
    female,
    alternative="two-sided"
)
p_value_mw

In [ ]:
alpha = 0.05

if p_value < alpha:
    print("Отвергаем H0: разница статистически значима")
else:
    print("Не можем отвергнуть H0: статистически значимой разницы нет")